In [1]:
print("\n\nம ஹா ம்ரி த் யு ஞ் ஜ யா\n\n")



ம ஹா ம்ரி த் யு ஞ் ஜ யா




In [2]:
#import tensorflow as tf

import pickle
import pandas as pd
import os
from matplotlib import pyplot
import numpy as np
import pandas as pd

from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score
#

In [3]:
def Box_filter(values, kernel_size=3):
    filter_values = np.cumsum(values, dtype=float)

    filter_values[kernel_size:] = filter_values[kernel_size:] - filter_values[:-kernel_size]
    filter_values[kernel_size:] = filter_values[kernel_size:] / kernel_size
    for i in range(1, kernel_size):
        filter_values[i] /= i + 1
    return filter_values

In [4]:
def extrapolate_next(values):
    last_value = values[-1]
    slope = [(last_value - v) / i for (i, v) in enumerate(values[::-1])]
    slope[0] = 0
    next_values = last_value + np.cumsum(slope)

    return next_values

In [5]:
def merge_series(values, extend_num=5, forward=5):

    next_value = extrapolate_next(values)[forward]
    extension = [next_value] * extend_num

    if isinstance(values, list):
        merge_values = values + extension
    else:
        merge_values = np.append(values, extension)
    return merge_values

In [6]:
class Saliency(object):
    def __init__(self, amp_window_size, series_window_size, score_window_size):
        self.amp_window_size = amp_window_size
        self.series_window_size = series_window_size
        self.score_window_size = score_window_size

    def frequency_domain_map(self, values):
        freq = np.fft.fft(values)
        mag = np.sqrt(freq.real ** 2 + freq.imag ** 2)
        spectral_residual = np.exp(np.log(mag) - Box_filter(np.log(mag), self.amp_window_size))
        freq.real = freq.real * spectral_residual / mag
        freq.imag = freq.imag * spectral_residual / mag
        freqmap = np.fft.ifft(freq)
        freq_map = np.sqrt(freqmap.real ** 2 + freqmap.imag ** 2)
        return freq_map

    def generate_anomaly_score(self, values, type="avg"):
        extended_series = merge_series(values, self.series_window_size, self.series_window_size)
        mag = self.frequency_domain_map(extended_series)[: len(values)]
        ave_filter = Box_filter(mag, self.score_window_size)        
        score = (mag - ave_filter) / ave_filter 
        return score

    def generate_anomaly_score2(self, values, type="avg"):
        extended_series = merge_series(values, self.series_window_size, self.series_window_size)
        mag = self.frequency_domain_map(extended_series)[: len(values)]
        return mag

    def generate_anomaly_score_(self, values, type="avg"):
        extended_series = merge_series(values, self.series_window_size, self.series_window_size)
        mag = self.frequency_domain_map(extended_series)[: len(values)]
        ave_filter = Box_filter(mag, self.score_window_size)
        ave_filter1 = Box_filter(ave_filter, 12)
        ave_filter2 = Box_filter(ave_filter1, 8)
        ave_filter = Box_filter(ave_filter2, 6)
        score = (mag - ave_filter) / ave_filter        
        return score
    

## Entire A1, A2, A3, A4 benchmark Yahoo opensource data in single folder

In [7]:
path = '/Users/manjunath.adinarayan/TIME-SERIES/YAHOO_Data/Full_data/'


In [8]:
import os
import glob

extension='csv'
os.chdir( path )
files = glob.glob('*.{}'.format(extension))
len(files)

367

In [9]:
# less than period
amp_window_size=3
# (maybe) as same as period
series_window_size=3
# a number enough larger than period
score_window_size=32

spec = Saliency(amp_window_size, series_window_size, score_window_size)

# ESR-3

# MULTIPLE FILTER LAYERS

# FULL- DATA [ Cold-start ]

In [10]:

prec_list, rec_list, f1score_list = [], [], []

for file in files:
    df = pd.read_csv(os.path.join(path,file), header=0)
    test_signal = df.value.values 
    score = spec.generate_anomaly_score_(test_signal)
#    index_changes = np.where(score > np.percentile(score, 99))[0]

    cutoff = 4.8
    pred_anom = []
    for i in range(0, len(score)):
        if score[i] > cutoff:
            pred_anom.append(1)
        else:
            pred_anom.append(0)
            
    if "Benchmark" in file:
        prec_list.append(precision_score(df.anomaly.values, pred_anom) )
        rec_list.append(recall_score(df.anomaly.values, pred_anom) )
        f1score_list.append(f1_score(df.anomaly.values, pred_anom) )
    else:
        prec_list.append(precision_score(df.is_anomaly.values, pred_anom) )
        rec_list.append(recall_score(df.is_anomaly.values, pred_anom) )
        f1score_list.append(f1_score(df.is_anomaly.values, pred_anom) )

np.mean(prec_list), np.mean(rec_list), np.mean(f1score_list)

/Users/manjunath.adinarayan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in double_scalars
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/manjunath.adinarayan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in long_scalars
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/manjunath.adinarayan/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples.
  'recall', 'true', average, warn_for)
/Users/manjunath.adinarayan/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true samples.
  'recall', 'true', average, warn_for)
/Users/manjunath.adinarayan/anaconda3/lib/python3.6/site-packag

(0.6972874391398184, 0.806273208068566, 0.6936235496074371)

# SECOND HALF OF DATA [ TEST DATA ]

In [11]:
prec_list, rec_list, f1score_list = [], [], []

for file in files:
    df1 = pd.read_csv(os.path.join(path,file), header=0)
    df = df1.iloc[int(len(df1)/2):, :]
    test_signal = df.value.values 
    score = spec.generate_anomaly_score_(test_signal)
    #index_changes = np.where(score > np.percentile(score, 99))[0]

    cutoff = 4.8
    pred_anom = []
    for i in range(0, len(score)):
        if score[i] > cutoff:
            pred_anom.append(1)
        else:
            pred_anom.append(0)
            
    if "Benchmark" in file:
        prec_list.append(precision_score(df.anomaly.values, pred_anom) )
        rec_list.append(recall_score(df.anomaly.values, pred_anom) )
        f1score_list.append(f1_score(df.anomaly.values, pred_anom) )
    else:
        prec_list.append(precision_score(df.is_anomaly.values, pred_anom) )
        rec_list.append(recall_score(df.is_anomaly.values, pred_anom) )
        f1score_list.append(f1_score(df.is_anomaly.values, pred_anom) )

np.mean(prec_list), np.mean(rec_list), np.mean(f1score_list)

/Users/manjunath.adinarayan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in double_scalars
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/manjunath.adinarayan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in long_scalars
  This is separate from the ipykernel package so we can avoid doing imports until


(0.7137222053382496, 0.7905652822820916, 0.6956508794793295)

# ESR-1

# Enhanced SR with no averaging/filtering of saliency map and cutoff = 7 * np.mean(score)

# FULL- DATA [ Cold-start ]

In [12]:
prec_list, rec_list, f1score_list = [], [], []

for file in files:
    df = pd.read_csv(os.path.join(path,file), header=0)
    test_signal = df.value.values 
    score = spec.generate_anomaly_score2(test_signal)
    #index_changes = np.where(score > np.percentile(score, 99))[0]
    
    cutoff = 7 * np.mean(score)
    pred_anom = []
    for i in range(0, len(score)):
        if score[i] > cutoff:
            pred_anom.append(1)
        else:
            pred_anom.append(0)
            
    if "Benchmark" in file:
        prec_list.append(precision_score(df.anomaly.values, pred_anom) )
        rec_list.append(recall_score(df.anomaly.values, pred_anom) )
        f1score_list.append(f1_score(df.anomaly.values, pred_anom) )
    else:
        prec_list.append(precision_score(df.is_anomaly.values, pred_anom) )
        rec_list.append(recall_score(df.is_anomaly.values, pred_anom) )
        f1score_list.append(f1_score(df.is_anomaly.values, pred_anom) )

np.mean(prec_list), np.mean(rec_list), np.mean(f1score_list)

/Users/manjunath.adinarayan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in double_scalars
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/manjunath.adinarayan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in long_scalars
  This is separate from the ipykernel package so we can avoid doing imports until


(0.6509540021083907, 0.7779652423245614, 0.6508797391830556)

# HALF OF THE DATA [ TEST DATA ]

In [13]:
prec_list, rec_list, f1score_list = [], [], []

for file in files:
    df1 = pd.read_csv(os.path.join(path,file), header=0)
    df = df1.iloc[int(len(df1)/2):, :]
    test_signal = df.value.values 
    score = spec.generate_anomaly_score2(test_signal)
    #index_changes = np.where(score > np.percentile(score, 99))[0]
    
    cutoff = 7 * np.mean(score)
    pred_anom = []
    for i in range(0, len(score)):
        if score[i] > cutoff:
            pred_anom.append(1)
        else:
            pred_anom.append(0)
            
    if "Benchmark" in file:
        prec_list.append(precision_score(df.anomaly.values, pred_anom) )
        rec_list.append(recall_score(df.anomaly.values, pred_anom) )
        f1score_list.append(f1_score(df.anomaly.values, pred_anom) )
    else:
        prec_list.append(precision_score(df.is_anomaly.values, pred_anom) )
        rec_list.append(recall_score(df.is_anomaly.values, pred_anom) )
        f1score_list.append(f1_score(df.is_anomaly.values, pred_anom) )

np.mean(prec_list), np.mean(rec_list), np.mean(f1score_list)

/Users/manjunath.adinarayan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in double_scalars
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/manjunath.adinarayan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in long_scalars
  This is separate from the ipykernel package so we can avoid doing imports until


(0.675088970218398, 0.7560466291248261, 0.6570270138255867)

# ESR-2

# Enhanced SR with averaging/filtering of saliency map

 # FULL- DATA [ Cold-start ]

In [14]:
prec_list, rec_list, f1score_list = [], [], []

for file in files:
    df = pd.read_csv(os.path.join(path,file), header=0)
    test_signal = df.value.values 
    score = spec.generate_anomaly_score(test_signal)
    #index_changes = np.where(score > np.percentile(score, 99))[0]
    cutoff = 2.8
    pred_anom = []
    for i in range(0, len(score)):
        if score[i] > cutoff:
            pred_anom.append(1)
        else:
            pred_anom.append(0)
            
    if "Benchmark" in file:
        prec_list.append(precision_score(df.anomaly.values, pred_anom) )
        rec_list.append(recall_score(df.anomaly.values, pred_anom) )
        f1score_list.append(f1_score(df.anomaly.values, pred_anom) )
    else:
        prec_list.append(precision_score(df.is_anomaly.values, pred_anom) )
        rec_list.append(recall_score(df.is_anomaly.values, pred_anom) )
        f1score_list.append(f1_score(df.is_anomaly.values, pred_anom) )

np.mean(prec_list), np.mean(rec_list), np.mean(f1score_list)

/Users/manjunath.adinarayan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in double_scalars
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/manjunath.adinarayan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in long_scalars
  This is separate from the ipykernel package so we can avoid doing imports until


(0.5968055594659185, 0.8690790440124405, 0.6590783629749551)

# HALF OF THE DATA [ TEST DATA ]

In [15]:
prec_list, rec_list, f1score_list = [], [], []

for file in files:
    df1 = pd.read_csv(os.path.join(path,file), header=0)
    df = df1.iloc[int(len(df1)/2):, :]
    test_signal = df.value.values 
    score = spec.generate_anomaly_score(test_signal)
    #index_changes = np.where(score > np.percentile(score, 99))[0]
    cutoff = 2.8
    pred_anom = []
    for i in range(0, len(score)):
        if score[i] > cutoff:
            pred_anom.append(1)
        else:
            pred_anom.append(0)
            
    if "Benchmark" in file:
        prec_list.append(precision_score(df.anomaly.values, pred_anom) )
        rec_list.append(recall_score(df.anomaly.values, pred_anom) )
        f1score_list.append(f1_score(df.anomaly.values, pred_anom) )
    else:
        prec_list.append(precision_score(df.is_anomaly.values, pred_anom) )
        rec_list.append(recall_score(df.is_anomaly.values, pred_anom) )
        f1score_list.append(f1_score(df.is_anomaly.values, pred_anom) )

np.mean(prec_list), np.mean(rec_list), np.mean(f1score_list)

/Users/manjunath.adinarayan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in double_scalars
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/manjunath.adinarayan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in long_scalars
  This is separate from the ipykernel package so we can avoid doing imports until


(0.6589798720628981, 0.8312036686714798, 0.6940316418644611)